In [3]:
import os
import json
import time
import docx
import fitz  # PyMuPDFをインポート ▼▼▼
import google.generativeai as genai
from dotenv import load_dotenv

# --- 準備 ---
load_dotenv()
try:
    api_key = os.environ.get("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("APIキーが.envファイルに設定されていません。")
    genai.configure(api_key=api_key)
except ValueError as e:
    print(e)
    exit()

# --- 関数定義 ---
def get_text_from_docx(filepath):
    """docxファイルからテキストを抽出する"""
    try:
        doc = docx.Document(filepath)
        return '\n'.join([para.text for para in doc.paragraphs])
    except Exception as e:
        print(f"  -> エラー: {filepath} の読み込みに失敗 - {e}")
        return ""

# ▼▼▼ PDFからテキストを抽出する関数を追加 ▼▼▼
def get_text_from_pdf(filepath):
    """PDFファイルからテキストを抽出する"""
    try:
        doc = fitz.open(filepath)
        text = ""
        for page in doc:
            text += page.get_text()
        doc.close()
        return text
    except Exception as e:
        print(f"  -> エラー: {filepath} の読み込みに失敗 - {e}")
        return ""

def get_evaluation_data(text_content):
    """Gemini APIを使用して論文の評価データを取得する"""
    # タイムアウトとリトライのロジックを簡素化
    model = genai.GenerativeModel('gemini-2.5-flash') # モデル名を修正
    prompt = f"""
    あなたは世界で一番優秀な論文査読者です。以下の論文本文を読み、8つの評価項目について、それぞれ1点から5点の整数で評価してください。
    評価の際は、必ずJSON形式で、各項目に`score`と、そのスコアに至った具体的な理由を50字以内で記述した`comment`を含めてください。
    他の説明は一切含めず、JSONオブジェクトのみを出力してください。

      # 評価項目とJSONキー (このキー名を厳守してください)
    - novelty: アイデアやアプローチの新規性
    - analysis_quality: 実験や分析の質の高さ
    - clarity: 文章や図表の分かりやすさ
    - experiment_validity: 実験計画や手法の妥当性
    - contribution: 研究分野への貢献度
    - structure: 論文全体の構成の巧みさ
    - comprehensiveness: 先行研究の調査など、研究の網羅性
    - depth_of_discussion: 結果に対する考察の深さや独創性

    # 論文本文
    {text_content}

    # 出力形式 (JSONのみ)
    {{
      "novelty": {{"score": (点数), "comment": "（コメント）"}},
      "analysis_quality": {{"score": (点数), "comment": "（コメント）"}},
      "clarity": {{"score": (点数), "comment": "（コメント）"}},
      "experiment_validity": {{"score": (点数), "comment": "（コメント）"}},
      "contribution": {{"score": (点数), "comment": "（コメント）"}},
      "structure": {{"score": (点数), "comment": "（コメント）"}},
      "comprehensiveness": {{"score": (点数), "comment": "（コメント）"}},
      "depth_of_discussion": {{"score": (点数), "comment": "（コメント）"}}
    }}
    """
    
    try:
        response = model.generate_content(prompt)
        cleaned_text = response.text.strip().replace('```json', '').replace('```', '').strip()
        return json.loads(cleaned_text)
    except Exception as e:
        print(f"  -> 評価データ取得中にエラー: {e}")
        return None

# --- 実行部分 ---
input_json_path = 'papers.json'
output_json_path = 'papers_with_eval.json'
papers_dir = './papers'

if not os.path.exists(input_json_path):
    print(f"エラー: 入力ファイル '{input_json_path}' が見つかりません。")
    exit()

with open(input_json_path, 'r', encoding='utf-8') as f:
    papers_data = json.load(f)

print("論文の評価スコアとコメントの生成を開始します...")

for paper in papers_data:
    # 既に新しい形式の評価データがある場合はスキップ
    if 'evaluation' in paper and isinstance(paper.get('evaluation', {}).get('novelty'), dict) and len(paper.get('evaluation', {})) == 8:
        print(f"{paper['filename']} は評価済みのためスキップします。")
        continue

    print(f"{paper['filename']} の評価データを生成中...")
    
    filepath = os.path.join(papers_dir, paper['filename'])
    
    # ▼▼▼ ファイルの拡張子によってテキスト抽出処理を分岐 ▼▼▼
    full_text = ""
    if filepath.lower().endswith('.docx'):
        full_text = get_text_from_docx(filepath)
    elif filepath.lower().endswith('.pdf'):
        full_text = get_text_from_pdf(filepath)
    
    if not full_text:
        print("  -> テキストが空のためスキップします。")
        continue

    evaluation_scores = get_evaluation_data(full_text)
    
    if evaluation_scores:
        paper['evaluation'] = evaluation_scores
        print("  -> データ生成完了。")
    else:
        # 失敗した場合も8項目分の空データを入れるように修正
        paper['evaluation'] = {
            "novelty": {"score": 0, "comment": "評価失敗"},
            "analysis_quality": {"score": 0, "comment": "評価失敗"},
            "clarity": {"score": 0, "comment": "評価失敗"},
            "experiment_validity": {"score": 0, "comment": "評価失敗"},
            "contribution": {"score": 0, "comment": "評価失敗"},
            "structure": {"score": 0, "comment": "評価失敗"},
            "comprehensiveness": {"score": 0, "comment": "評価失敗"},
            "depth_of_discussion": {"score": 0, "comment": "評価失敗"}
        }
        print("  -> データ生成に失敗しました。")

    # APIのレート制限を避けるための待機
    time.sleep(1) 

with open(output_json_path, 'w', encoding='utf-8') as f:
    json.dump(papers_data, f, ensure_ascii=False, indent=2)

print(f"\n全ての論文の評価が完了し、'{output_json_path}'に保存しました。")

論文の評価スコアとコメントの生成を開始します...
02_内山田湖太_卒論_最終版.docx の評価データを生成中...
  -> データ生成完了。
2005 matsumoto yoo hirayama petrova.pdf の評価データを生成中...
  -> データ生成完了。
CV_20250708_2024000371.pdf の評価データを生成中...
  -> データ生成完了。
EI_16_田中慶人_卒論v4.docx の評価データを生成中...
  -> データ生成完了。
hujii.pdf の評価データを生成中...
  -> データ生成完了。
ito.pdf の評価データを生成中...
  -> データ生成完了。
kaku.docx の評価データを生成中...
  -> データ生成完了。
kazuki.docx の評価データを生成中...
  -> データ生成完了。
mae.docx の評価データを生成中...
  -> データ生成完了。
naka.docx の評価データを生成中...
  -> データ生成完了。
oisi - コピー - コピー.docx の評価データを生成中...
  -> データ生成完了。
oisi - コピー.docx の評価データを生成中...
  -> データ生成完了。
oisi.docx の評価データを生成中...
  -> データ生成完了。
soma - コピー.docx の評価データを生成中...
  -> データ生成完了。
soma.docx の評価データを生成中...
  -> データ生成完了。
The Japanese Journal of Health Psychology, 2007, Vol.20, No.2, 30-41.pdf の評価データを生成中...
  -> データ生成完了。
自分の卒論.docx の評価データを生成中...
  -> データ生成完了。

全ての論文の評価が完了し、'papers_with_eval.json'に保存しました。
